# Introduction to Search Relevance Models

*Note: This is the companion notebook to the [Introduction to Search Relevance Models](https://lemmalytica.com/posts/2020/10/13/search-relevance-models/) article on [Lemmalytica](https://lemmalytica.com).*

One of the core tasks in information retrieval is searching. Anyone who deals with large amounts of text data (and that's almost all of us) knows how difficult this seemingly simple task can be. If your search term is too broad, you may find yourself sifting through an impossible quantity of documents. And if your search term is too narrow, you are likely to miss out on important documents. So how do we decide which documents are most relevant to our search?

Search relevance is a difficult problem--and modern search engines employ highly sophisticated (and proprietary) algorithms to deal with the issue. We won't delve into those algorithms, but let's look at some simple strategies that you might employ in your own information retrieval applications.

## State of the Union

As a sample dataset, we will be using the text of State of the Union (SotU) addresses by U.S. Presidents. Imagine that you're a historian studying different policy priorities in U.S. history. How would you decide which SotU address was most related to your topic of interest? For example, say you are interested in taxes. Is a simple search in the corpus for "taxes" enough? Will that lead to too many results and information overload? Let's find out!

To get started, we will load the libraries we are going to use and read our data into a dataframe.

In [1]:
# import utility and data libraries
import os
import re
import pandas as pd
import numpy as np

# import NLP libraries
import spacy
from gensim.corpora.dictionary import Dictionary

In [2]:
# load state of the union texts
def load_texts(dir_path):
    """
    - Parameters: dir_path (string) for a directory containing text
      files.
    - Returns: A list of dictionaries with keys file_name and text.
    """
    docs = []
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        if file_name.endswith(".txt") and os.path.isfile(file_path):
            with open(file_path, "r+", encoding="utf-8") as file:
                text = file.read()
                current = {
                    "file_name": file_name,
                    "text": text
                }
                docs.append(current)
    return docs

def add_sotu_metadata(sotu_doc_dict):
    """
    - Parameters: sotu_doc_dict (dictionary) with sotu metadata.
      Expects a file_name key in format "president_year.txt"
    - Returns: A dictionary with appended president and year keys.
    """
    file_name = sotu_doc_dict["file_name"]
    pres, year, filetype = re.split(r"[^A-Za-z0-9]", file_name)
    sotu_doc_dict["president"] = pres
    sotu_doc_dict["year"] = int(year)
    return sotu_doc_dict

def load_sotu_texts(dir_path):
    """
    - Parameters: dir_path (string) for a directory containing text
      files.
      Expects sotu text files in dir_path in format "president_year.txt".
    - Returns: A Pandas DataFrame with file_name, text, president, and
      year columns for each sotu text in dir_path.
    """
    docs = load_texts(dir_path)
    docs = [add_sotu_metadata(d) for d in docs]
    docs = sorted(docs, key=lambda d: d["year"])
    df = pd.DataFrame(docs)
    return df[["year", "president", "text"]]

sotu_df = load_sotu_texts("data")

### Exploring Data

As with any data project, we should start by looking at our data and seeing what we can learn about it.

In [10]:
print(f"Total rows: {sotu_df.shape[0]}")
print(f"Date range: {min(sotu_df['year'])} to {max(sotu_df['year'])}")
print("Initial rows:")
display(sotu_df.head())


Total rows: 228
Date range: 1790 to 2018
Initial rows:


,year,president,text
0,1790,Washington,"Fellow Citizens of the Senate, and House of Re..."
1,1791,Washington,Fellow-Citizens of the Senate and House of Rep...
2,1792,Washington,Fellow-Citizens of the Senate and House of Rep...
3,1793,Washington,Fellow-Citizens of the Senate and House of Rep...
4,1794,Washington,Fellow-Citizens of the Senate and House of Rep...


It looks like we have 228 observations (that is, different SotU speeches), each with `year`, `president`, and `text` columns. Also, it seems like Washington liked to start all of his speeches the exact same way!

## Simple Search

Now that we have our data, let's start searching for specific terms. At its most basic, a search query asks the question "which documents contain all of the terms in this query?" This is a simple strategy and we can use it to search through the `text` column of our dataframe. 

Putting on our historian hat, let's imagine that we are interested in comparing technology and tax policy. As part of our work, we want to find all of the SotU addresses that discuss both issues. To do this, we will use the search query "technology taxes". For the sake of simplicity, we will treat this as a boolean `and` query.

First, we will need to split our query into individual tokens and then filter our dataframe for all of the observations where the `text` column has at least one occurrence of each of the terms in our query.

In [46]:
# simple boolean search
def search_df_texts(df, query_string: str):
    """
    - Parameters: df (Pandas DataFrame), query_string (string). df must 
      contain a "text" column.
    - Returns: A subset of df containing only rows where each term in 
      query_string appeared as a substring in df["text"].
    """
    terms = query_string.lower().split(" ")
    filters = [df["text"].str.lower().str.contains(term)
               for term in terms]
    return df[np.all(filters, axis=0)]

search_term = "technology taxes"
results = search_df_texts(sotu_df, search_term)

print(f"Num results for query '{search_term}': {results.shape[0]}\n")
results.head()

Num results for query 'technology taxes': 38



,year,president,text
116,1906,Roosevelt,To the Senate and House of Representatives:\n\...
165,1956,Eisenhower,To the Congress of the United States:\n\nThe o...
170,1961,Eisenhower,To the Congress of the United States:\n\nOnce ...
174,1965,Johnson,"On this Hill which was my home, I am stirred b..."
176,1967,Johnson,"Mr. Speaker, Mr. Vice President, distinguished..."


Great! We have 38 results that include both the term `technology` and the term `taxes`. But how do we know where to start? Our dataframe was already sorted in chronological order, which is also how the search results are sorted. However, this tells us nothing about which of the SotU speeches is *most* relevant to our search. We're going to have to read through all 38 speeches to see which ones are the most useful. There must be a better way!

## Bag-of-Words and the Term-Document Matrix

To improve on our simple search model, we need some kind of metric to use when comparing search results. We already know that all of our search results contain each of our search terms at least once, so what else can we do? The most obvious solution is to look at word frequency. If one document contains our search terms many times, and another document just a few times, then the first one is likely more relevant.

To look at word frequency, we can use the so-called [*bag-of-words*](https://lemmalytica.com/posts/2020/10/11/bag-of-words/) (BoW) model. A BoW turns a document into a collection of token-frequency tuples. We can view this collection as a vectorized version of our text, which is useful in various machine learning tasks. For our purposes though, we aren't feeding the model into an algorithm--rather, we want to use it as a metric when comparing search results.

To use the BoW as a search relevance metric, we can convert it into a `term-document matrix`. A term-document matrix is a table where each row is a document and each column is a token. The values in the cells are the number of times a particular token occurs in each document. Using this, we can sort our search results by the count of a particular token (or the sum of the counts of all the terms in our search).

### Data Preparation

To get started, we will need to tokenize our documents and build a *vocabulary* from the overall collection. A vocabulary is the set of unique tokens in our text and will be used as the columns in our term-document matrix. To carry out these tasks, we can use [*spaCy*](https://spacy.io/) for tokenization and [*Gensim*](https://radimrehurek.com/gensim/index.html) to build a vocabulary (aka *dictionary*).

In [13]:
# load spaCy model
nlp = spacy.load("en_core_web_md")

# tokenize documents
def spacy_doc(model, text, lower=True):
    """
    - Parameters: model (spaCy model), text (string), lower (bool).
    - Returns: A spaCy Document object processed using the provided
      model. Document is all lowercase if lower is True.
    """
    if lower:
        text = text.lower()
    return model(text)

sotu_docs = [spacy_doc(nlp, text) for text in sotu_df["text"]]

# build dictionary
def get_token_texts(doc):
    """
    - Parameters: doc (spaCy Document object).
    - Returns: A list of strings based on the text value of each token
      in doc.
    """
    token_list = [token for token in doc]
    return [token.text for token in token_list]

def build_dictionary(doc_list):
    """
    - Parameters: doc_list (list of spaCy Document objects).
    - Returns: A Gensim Dictionary, built using the tokens in each
      document contained in doc_list.
    """
    return Dictionary([get_token_texts(doc) for doc in doc_list])

sotu_dictionary = build_dictionary(sotu_docs)

### Building a Term-Document Matrix

Now that we have our tokenized documents and our corpus vocabulary, we can build our BoW representations for each text and use those to create term-document matrices. Of note, Gensim `Dictionary` objects represent each token by a numerical id rather than the token text. Since we're interested in manually inspecting our term-document matrices, we should re-label the columns with the actual token texts and convert the matrix into a dataframe for easy manipulation.

In [14]:
# build bag-of-words representations and a term-document matrix
def build_corpus(doc_list, dictionary):
    """
    - Parameters: doc_list (list of spaCy Document objects), dictionary
      (Gensim Dictionary object).
    - Returns: A list of documents in bag-of-words format, containing
      tuples with (token_id, token_count) for each token in the text.
    """
    return [dictionary.doc2bow(get_token_texts(doc)) for doc in doc_list]

def build_td_matrix(doc_list, dictionary):
    """
    - Parameters: doc_list (list of spaCy Document objects), dictionary
      (Gensim Dictionary object).
    - Returns: A term-document matrix in the form of a 2D NumPy Array,
      where each row contains the count of a token in the corresponding
      document and each column index is the id of a token in the
      dictionary.
    """
    corpus = build_corpus(sotu_docs, sotu_dictionary)
    tdm = []
    for bow in corpus:
        vector = np.zeros(len(dictionary))
        for token_id, token_count in bow:
            vector[token_id] = token_count
        tdm.append(vector)
    return np.array(tdm)

def build_term_document_df(doc_list, dictionary):
    """
    - Parameters: doc_list (list of spaCy Document objects), dictionary
      (Gensim Dictionary object).
    - Returns a term-document matrix in the form of a Pandas Dataframe, 
      where each row is a document and each column is a token. Values in
      the dataframe are token counts for the given document / token.
    """
    tdm = build_td_matrix(doc_list, dictionary)
    cols = list(dictionary.token2id.keys())
    return pd.DataFrame(tdm, columns=cols, dtype=pd.Int64Dtype)

sotu_td_df = build_term_document_df(sotu_docs, sotu_dictionary)

### Exploring the Term-Document Matrix

Per usual, now that we have a new dataframe we had better take a look at it!

In [25]:
print(f"Number of documents: {sotu_td_df.shape[0]}")
print(f"Number of terms: {sotu_td_df.shape[1]}\n")
print(f"Sample observations:")
sotu_td_df.iloc[:5, 100:105]

Number of documents: 228
Number of terms: 28393

Sample observations:


,contributes,convenience,convey,convincing,cool
0,1,1,1,1,1
1,1,2,0,0,0
2,0,0,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0


As expected, we still have 228 documents. Each document is an observation (row) in the matrix and has 28,393 columns. Wow! That's a lot of columns (which is why we subsetted to look at only a few of them). Each entry then contains the frequency count for a given token in each document. Of course, this is a *sparse matrix*, meaning that the vast majority of entries are zero (because most SotU speeches will not use every word in the vocabulary--otherwise it would be a long speech!)

It's important to note that the order and indices of these documents match the order in our `sotu_df` dataframe from earlier, which contains the text and metadata about each SotU speech. We had better not change the indices otherwise we won't be able to join them later.

### Searching a Term-Document Matrix

It looks like we're in good shape. Now that we have a term-document matrix, we have a metric that we can use when assessing search relevance. We will still filter our `sotu_df` dataframe for those rows that contain our search terms in the `text` column, except now we can join the dataframe with the term frequency counts from our term-document matrix and sort accordingly. Let's see what happens with our earlier search term of "technology taxes".

In [28]:
# term-document frequency search based on the bag-of-words model
def search_td_df(td_df, text_df, query_string: str):
    """
    - Parameters: td_df (Pandas DataFrame) representing a term-document
      matrix, text_df (Pandas DataFrame) with a "text" column and rows 
      that correspond to the td_df, and query_string (string).
    - Returns: A new dataframe that only contains rows from text_df where
      the "text" column had at least one occurence of each term in
      query_string. Additional columns are added to show the count of
      each term and the total count of all terms.
    """
    terms = query_string.lower().split(" ")
    filters = [td_df[term] > 0 for term in terms]
    filtered_td_df = td_df[np.all(filters, axis=0)][terms]
    filtered_td_df["terms_sum"] = filtered_td_df.agg(sum, axis=1) \
        .astype("int64")
    full_df = text_df.merge(filtered_td_df, 
        left_index=True, right_index=True)

    return full_df.sort_values("terms_sum", ascending=False)

results = search_td_df(sotu_td_df, sotu_df, search_term)

print(f"Num results for query '{search_term}': {results.shape[0]}\n")
results.head()

Num results for query 'technology taxes': 38



,year,president,text,technology,taxes,terms_sum
184,1975,Ford,"Mr. Speaker, Mr. Vice President, Members of th...",2,10,12
221,2012,Obama,"Mr. Speaker, Mr. Vice President, members of Co...",3,8,11
181,1972,Nixon,"Mr. Speaker, Mr. President, my colleagues in t...",6,5,11
217,2008,Bush,"Madam Speaker, Vice President Cheney, Members ...",5,6,11
213,2004,Bush,"Mr. Speaker, Vice President Cheney, Members of...",1,9,10


Already we can see that searching our term-document matrix produced more interesting results than our simple search. We can see term counts for each of our query terms as well as a sum of all the counts together. Given that "technology" and "taxes" are mentioned more frequently in these documents, they're probably a good place to start.

Of course, there are more nuanced ways to look at the search relevance problem. To finish up our introduction to search relevance, we'll look at one more model.

## Term Frequency-Inverse Document Frequency

Term frequency alone is a reasonable metric for search relevance, but imagine a case where you are searching for a term that is relatively common across the entire set of documents. Perhaps the term "government" or "america" in the case of our corpus of SotU addresses. Nearly all of the documents are likely to mention the terms. Some may use the word "government" more often than others, but because of how common the term is across all the documents, it starts to lose its meaning.

One way of addressing this problem is the [*term frequency-inverse document frequency*](https://lemmalytica.com/posts/2020/10/11/tf-idf/) (TF-IDF) model. Rather than use term frequency as our metric, we can use TF-IDF to give more weight to rare terms over common terms. We do this by calculating the document frequency (number of documents that include a term at least once) and the inverse document frequency (log of N / document frequency, where N is the total number of documents) and then multiplying it by the term frequency.

The result is a measure that still takes into account the number of times a term appears in a document, but gives priority to those terms that are generally rare. Let's take a look at TF-IDF in action.


In [29]:
# build tf-idf model
def document_frequency(td_df, term: str):
    """
    - Parameters: td_df (Pandas DataFrame) representing a term-document
      matrix, and term (string).
    - Returns: The document frequency value showing the number of
      documents in td_df where term occurs at least once.
    """
    return td_df[td_df[term] > 0].shape[0]

def inverse_document_frequency(td_df, term: str):
    """
    - Parameters: td_df (Pandas DataFrame) representing a term-document
      matrix, and term (string).
    - Returns: The inverse document frequency value for term, calculated
      as N / log(dft) where N is the number of documents in td_df and
      dft is the document frequency value for term.
    """
    N = td_df.shape[0]
    dft = document_frequency(td_df, term)
    return (N / np.log10(dft))
    
def build_tfidf_df(td_df):
    """
    - Parameters: td_df (Pandas DataFrame) representing a term-document
      matrix.
    - Returns: Returns a term frequency-inverse document frequency
      (TF-IDF) matrix in the form of a Pandas DataFrame, where each row
      is a document and each column is a token. Values in the dataframe
      are TF-IDF values for the given document / token.
    """
    def calculate_tfidf(col, td_df):
        idf = inverse_document_frequency(td_df, col.name)
        return col * idf
    
    return td_df.apply(calculate_tfidf, td_df=td_df)

sotu_tfidf_df = build_tfidf_df(sotu_td_df)

### Exploring the TF-IDF Matrix

Before looking at the TF-IDF matrix, let's see how some IDF scores compare for a relatively common word in the corpus like "government" and a rare one like "moon".

In [42]:
def print_idf_info(td_df, word):
    num_docs = sotu_td_df.shape[0]
    df = document_frequency(td_df, word)
    idf = inverse_document_frequency(td_df, word)
    print(f"'{word}' is in {df}/{num_docs} documents with IDF of {idf}.")

print_idf_info(sotu_td_df, "government")
print_idf_info(sotu_td_df, "moon")

'government' is in 227/228 documents for an IDF of 96.7731314594443.
'moon' is in 12/228 documents for an IDF of 211.2712770306409.


As expected, "government" appears in almost every document and thus has a modest IDF score of 96.77, whereas "moon" appears in only 12/228 documents and thus has a high IDF score of 211.27. If we were to search our TF-IDF matrix for the terms "government moon" then appearances of the term "moon" would carry more than twice as much weight as appearances of the term "government".

Our project is focused on the terms "technology" and "taxes", neither of which is quite as common as "government" or as rare as "moon", but the weights will nonetheless differ. Before moving on, let's see what those weights are.

In [44]:
print_idf_info(sotu_td_df, "technology")
print_idf_info(sotu_td_df, "taxes")

'technology' is in 50/228 documents for an IDF of 134.19895549545362.
'taxes' is in 131/228 documents for an IDF of 107.68577483094036.


### Searching the TF-IDF Matrix

With the above in mind, let's see how our TF-IDF search differs from the term-document matrix search from earlier.

In [43]:
# search based on the tf-idf model
def search_tfidf_df(tfidf_df, text_df, query_string: str):
    """
    - Parameters: tfidf_df (Pandas DataFrame) representing a tf-idf
      matrix, text_df (Pandas DataFrame) with a "text" column and rows
      that correspond to the tfidf_df, and query_string (string).
    - Returns: A new dataframe that only contains rows from text_df where
      the corresponding tf-idf value was greater than zero for each of
      the terms in query_string. Additional columns are added to show the
      tf-idf value for each term and the sum of the tf-idf values. 
    """
    terms = query_string.lower().split(" ")
    filters = [tfidf_df[term] > 0 for term in terms]
    filtered_tfidf_df = tfidf_df[np.all(filters, axis=0)][terms]
    filtered_tfidf_df["tfidf_sum"] = filtered_tfidf_df.agg(sum, axis=1)
    full_df = text_df.merge(filtered_tfidf_df, 
                            left_index=True, right_index=True)

    return full_df.sort_values("tfidf_sum", ascending=False)

results = search_tfidf_df(sotu_tfidf_df, sotu_df, search_term)

print(f"Num results for query '{search_term}': {results.shape[0]}\n")
results.head()

Num results for query 'technology taxes': 38



,year,president,text,technology,taxes,tfidf_sum
184,1975,Ford,"Mr. Speaker, Mr. Vice President, Members of th...",268.398,1076.86,1345.255659
181,1972,Nixon,"Mr. Speaker, Mr. President, my colleagues in t...",805.194,538.429,1343.622607
217,2008,Bush,"Madam Speaker, Vice President Cheney, Members ...",670.995,646.115,1317.109426
220,2011,Obama,"Mr. Speaker, Mr. Vice President, members of Co...",1073.59,215.372,1288.963194
221,2012,Obama,"Mr. Speaker, Mr. Vice President, members of Co...",402.597,861.486,1264.083065


In our TF-IDF model, the word "technology" carried about 25% more weight than the word "taxes". And indeed, it seems to have affected the results! The rank order of the top five results from the two searches is as follows.

In [81]:
td_results = search_td_df(sotu_td_df, sotu_df, search_term) \
    .head(5).reset_index(drop=True)
td_results["term_frequency_search"] = td_results["year"].astype(str) \
    + " - " + td_results["president"]

tfidf_results = search_tfidf_df(sotu_tfidf_df, sotu_df, search_term) \
    .head(5).reset_index(drop=True)
tfidf_results["tfidf_search"] = tfidf_results["year"].astype(str) \
    + " - " + tfidf_results["president"]

top_5 = td_results[["term_frequency_search"]].merge(
    tfidf_results[["tfidf_search"]], 
    left_index=True, 
    right_index=True
)

top_5

,term_frequency_search,tfidf_search
0,1975 - Ford,1975 - Ford
1,2012 - Obama,1972 - Nixon
2,1972 - Nixon,2008 - Bush
3,2008 - Bush,2011 - Obama
4,2004 - Bush,2012 - Obama


The results may not seem all that different--but imagine that you are working with a significantly larger data set. Weighting your search with TF-IDF may prove decisive in your effort to find the most relevant search results.

## Challenges

Our goal in comparing simple searches, term frequency searches, and TF-IDF searches isn't necessarily to determine which is best. Rather, the goal has been to see how different methods of information retrieval produce different results. Which model you use will depend in part on your goals.

Of course, our basic introduction ignores a lot of important issues. For example, should term proximity matter? Should we only return results that include all search terms (as we chose to do here) or should we also return results for partial matches? What about related words? Should a search for "taxes" also return results for "tax" and "taxation"? Or perhaps we shouldn't focus on words at all, but instead focus on themes. Perhaps a search for "technology" should also return documents that discuss "innovation" and "science". The list of possible refinements goes on.

Search relevance, and information retrieval more broadly, is a complex and fascinating topic. If you're interested in learning more, [Introduction to Information Retrieval](https://www.amazon.com/gp/product/0521865719/ref=as_li_tl?ie=UTF8&tag=lemmalytica-20&camp=1789&creative=9325&linkCode=as2&creativeASIN=0521865719&linkId=f0a6d49537ea9a727debd37dd9e52383) by Christopher D. Manning is arguably the seminal book on the topic and well worth a read.

Hopefully this has been an interesting introduction. Stay tuned for more articles on how to explore your text data sets!
